## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import keras

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


The features of this model will be APPLICATION_TYPE, AFFILIATION, CLASSIFICATION, USE_CASE, ORGANIZATION, STATUS, INCOME_AMT, SPECIAL_CONSIDERATIONS, and ASK_AMT.

The target of this model will be IS_SUCCESSFUL.

In [2]:
application_df.dtypes

EIN                        int64
NAME                      object
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_prep = application_df.drop(['EIN', 'NAME'], axis = 1)

In [4]:
# Determine the number of unique values in each column.
application_prep.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

Categorical Columns with more than 10 unique values: APPLICATION_TYPE (17), CLASSIFICATION (71)

In [5]:
# Look at APPLICATION_TYPE value counts for binning
application_types = pd.Series(application_prep['APPLICATION_TYPE']).value_counts()
application_types

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced - Starter_Code cutoff was between 156 and 528
application_cutoff = 200 # Number of applications for that type

# use the variable name `application_types_to_replace`
application_types_to_replace = []

for app, count in application_types.iteritems():
    if count < application_cutoff :
        application_types_to_replace.append(app)

# Replace in dataframe
for app in application_types_to_replace:
    application_prep['APPLICATION_TYPE'] = application_prep['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_prep['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
classification_types = pd.Series(application_prep['CLASSIFICATION']).value_counts()
classification_types

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_types[classification_types > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced - Starter_Code cutoff was between 1883 and 777
classification_cutoff = 1000

# use the variable name `classifications_to_replace`
classifications_to_replace = []

for classification, count in classification_types.iteritems():
    if count < classification_cutoff :
        classifications_to_replace.append(classification)

# Replace in dataframe
for cls in classifications_to_replace:
    application_prep['CLASSIFICATION'] = application_prep['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_prep['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummies = pd.get_dummies(application_prep)

In [11]:
# Split our preprocessed data into our features and target arrays
X = application_dummies.drop(['IS_SUCCESSFUL'], axis = 1)
y = application_dummies['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

### Tutor Question: How to get the correct number of parameters?

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
num_features = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=num_features))

# Second hidden layer ("If Necessary")
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 80)                3520      
_________________________________________________________________
dense_1 (Dense)              (None, 30)                2430      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 31        
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [33]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE

# Second hidden layer
#  YOUR CODE GOES HERE

# Output layer
#  YOUR CODE GOES HERE

# Check the structure of the model
nn.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 80)                3520      
_________________________________________________________________
dense_1 (Dense)              (None, 30)                2430      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 31        
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [34]:
# Create a callback that saves the model's weights every 5 epochs.
batch_size = 30
mc = keras.callbacks.ModelCheckpoint('weights{epoch:08d}.h5', save_weights_only=True, save_freq=5*batch_size, verbose = 2)

In [35]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs=50, callbacks=[mc])

Epoch 1/50
115/804 [===>..........................] - ETA: 0s - loss: 0.5346 - accuracy: 0.7402
Epoch 00001: saving model to weights00000001.h5
214/804 [======>.......................] - ETA: 0s - loss: 0.5294 - accuracy: 0.7395
Epoch 00001: saving model to weights00000001.h5
442/804 [===============>..............] - ETA: 0s - loss: 0.5294 - accuracy: 0.7436
Epoch 00001: saving model to weights00000001.h5
548/804 [===================>..........] - ETA: 0s - loss: 0.5293 - accuracy: 0.7429
Epoch 00001: saving model to weights00000001.h5
648/804 [=======================>......] - ETA: 0s - loss: 0.5310 - accuracy: 0.7419
Epoch 00001: saving model to weights00000001.h5
804/804 [==============================] - 0s 470us/step - loss: 0.5325 - accuracy: 0.7409
Epoch 2/50
  1/804 [..............................] - ETA: 0s - loss: 0.5267 - accuracy: 0.7500
Epoch 00002: saving model to weights00000002.h5
194/804 [======>.......................] - ETA: 0s - loss: 0.5257 - accuracy: 0.7463
Epoc

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5583 - accuracy: 0.7286
Loss: 0.5583301186561584, Accuracy: 0.7286297082901001


In [37]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/8575 - 0s - loss: 0.5578 - acc: 0.7263
Loss: 0.557812534073699, Accuracy: 0.7262973785400391


In [25]:
nn.save("fit_model.h5")

In [26]:
from tensorflow.keras.models import load_model
voice_model = load_model("fit_model.h5")

In [28]:
model_loss, model_accuracy = voice_model.evaluate(
    X_test_scaled, y_test, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5583 - accuracy: 0.7286
Normal Neural Network - Loss: 0.5583301186561584, Accuracy: 0.7286297082901001
